# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sqlalchemy
from sqlalchemy import create_engine
import pandas as pd

# NLP
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import re
import nltk
nltk.download(['stopwords','punkt', 'wordnet', 'averaged_perceptron_tagger'])

# DS
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
!pip install optuna
import optuna
import xgboost as xgb
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import cross_val_score


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.1 MB/s eta 0:00:00


In [2]:
# load data from database
engine = create_engine('sqlite:///df_clean.db')
df = pd.read_sql_table("disaster_messages", con=engine.connect())
X = df.message
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # normalize
    re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    # tokenize
    words = word_tokenize(text)

    # stop word removal
    words = [w for w in words if w not in stopwords.words("english")]

    # Lemmatization
    tokens = [WordNetLemmatizer().lemmatize(w) for w in words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([('vect',CountVectorizer(tokenizer=tokenize)),
                    ('tfidf',TfidfTransformer()),
                    ('rf',MultiOutputClassifier(RandomForestClassifier(random_state=42)))
                   ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

### 5. Test your model


In [ ]:
pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)

In [8]:
(y_pred==y_test).mean().mean()

0.947929909314349

### 6. Improve your model
Use optuna library to find better parameters.

In [10]:
pipeline = Pipeline([('vect',CountVectorizer(tokenizer=tokenize)),
                    ('tfidf',TfidfTransformer()),
                    ('rf',MultiOutputClassifier(RandomForestClassifier()))
                   ])

param_distributions = {
    'rf__estimator__n_estimators': optuna.distributions.IntDistribution(5, 100, log=False, step=5),
    'rf__estimator__min_samples_leaf': optuna.distributions.IntDistribution(1, 9,log=False,step=2),
    'rf__estimator__random_state':optuna.distributions.CategoricalDistribution([42])
}

optuna_search = optuna.integration.OptunaSearchCV(pipeline, param_distributions)
optuna_search.fit(X_train,y_train)
y_pred = optuna_search.predict(X_test)
print((y_pred==y_test).mean().mean())


<ipython-input-10-7278962f16c0>:12: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(pipeline, param_distributions)
[I 2023-08-26 18:52:51,189] A new study created in memory with name: no-name-84f1dd7d-5163-4fa9-a39a-645e6bc0cfe5
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserW

0.9481841681498433


In [11]:
pipeline = Pipeline([('vect',CountVectorizer(tokenizer=tokenize)),
                    ('tfidf',TfidfTransformer()),
                    ('rf',MultiOutputClassifier(RandomForestClassifier(n_estimators=95,min_samples_leaf=1,random_state=42)))
                   ])


In [ ]:
pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
(y_pred==y_test).mean().mean()

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
def test_model(y_test, y_pred):

    """
    Function to iterate through columns and call sklearn classification report on each.
    """
    for index, column in enumerate(y_test):
        print(column, classification_report(y_test[column], y_pred[:, index]))

test_model(y_test,y_pred)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
import pickle
filename = 'model.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.